In [25]:
# --------------------------------------------------------------------------

# ----------------- simpls models after sample_selection functions

# -------------------------------------------------------------------------


# ¡¡¡ --- !!! # ---> modules and data cases

# --- system modules

import sys
import datetime
import os


base_dir = "/home/u0106869/vfonsecad/kul_phd/github_projects/sample_selection_simpls"

# --- data handling modules

import numpy as np
import pandas as pd
import scipy.io as sp_io
import scipy as sp

# --- visualization modules

import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- my modules

methods_dir = base_dir + '/methodology'  
sys.path.insert(0, methods_dir + '/model_building')
sys.path.insert(0, methods_dir + '/read_data')
sys.path.insert(0, methods_dir + '/sample_selection')
from class_chemometrics_data import chemometrics_data
from class_sample_selection import sample_selection
import simpls_module




# ¡¡¡ --- !!! # ---> base working directory and available data cases


# ************************************ init --- user 
cases_dict = {"d0001": ["d0001_corn", "d0001_data_prepared_01"]}
# ************************************ end --- user 


print("--------- imports loaded ----------")


experimentID = "exp001"

# ¡¡¡ --- !!! # ---> data


# ************************************ init --- user 
caseID_key = "d0001"
# ************************************ end --- user 

case_dir = cases_dict[caseID_key][0]
dname = cases_dict[caseID_key][1]
data_dir = '/data/' + case_dir + '/data_prepared/'





# ************************************ init --- user
data_class = chemometrics_data(base_dir + data_dir + dname + '.mat', 
                               data_identifier = data_dir + dname,
                               include_val = False,
                               include_test = True,
                               include_unlabeled = False,
                               y_all_range = False,
                               y_range = np.array([0]),
                               obs_all_cal = True,
                              shuffle = False)
# ************************************ end --- user



print(data_class.ncal, data_class.K)
print(data_class.get_test()["xtest"].shape)


print("--------- data loaded for " + data_class.data_identifier + "----------")




--------- imports loaded ----------
56 700
(24, 700)
--------- data loaded for /data/d0001_corn/data_prepared/d0001_data_prepared_01!*moisture*!----------


In [26]:
# --- initialize numba functions

output_pls = simpls_module.simpls_fit(xx=data_class.get_cal()["xcal"], yy=data_class.get_cal()["ycal"], nlv=14)
ytest_pred = simpls_module.simpls_predict(data_class.get_test()["xtest"],  output_pls[0],output_pls[1],output_pls[2])
rmsep = simpls_module.rmse(data_class.get_test()["ytest"], ytest_pred, np.ones(ytest_pred.shape[0]))
r2 = simpls_module.r2(data_class.get_test()["ytest"], ytest_pred, np.ones(ytest_pred.shape[0]))
cv_output = simpls_module.simpls_univariate_cv(xx=data_class.get_cal()["xcal"], yy=data_class.get_cal()["ycal"], total_nlv=40, number_splits=10, number_repetitions=4)


print("done")

done


In [27]:
# --- functions for model performance

def model_performance_cv_val_test(X0,Y0,selected_rows,Xtest, Ytest,total_lv=25, cv_reps = 2):



    Xc = X0 - X0.mean(axis=0)
    Yc = Y0 - Y0.mean(axis=0)  


    # ¡¡¡ --- !!! get samples

    cal_samples = selected_rows.copy()
    n_ss = cal_samples.sum()
 


    # --- train, cv, val, test

    Xcal = X0[cal_samples==1,:]
    Ycal = Y0[cal_samples==1,:]
    Xval = X0[cal_samples==0,:]
    Yval = Y0[cal_samples==0,:]  
    
    
    trained_pls = simpls_module.simpls_fit(xx=Xcal, yy=Ycal, nlv=total_lv)
    yval_pred = simpls_module.simpls_predict(Xval,  trained_pls[0],trained_pls[1],trained_pls[2])
    ytest_pred = simpls_module.simpls_predict(Xtest,  trained_pls[0],trained_pls[1],trained_pls[2])
    
    
    rmseval = simpls_module.rmse(Yval, yval_pred, np.ones(yval_pred.shape[0]))[:,0]
    rmsep = simpls_module.rmse(Ytest, ytest_pred, np.ones(ytest_pred.shape[0]))[:,0]
    
    r2val = simpls_module.r2(Yval, yval_pred, np.ones(yval_pred.shape[0]))[:,0]
    r2p = simpls_module.r2(Ytest, ytest_pred, np.ones(ytest_pred.shape[0]))[:,0]
    
    
    cv_output = simpls_module.simpls_univariate_cv(xx=Xcal, yy=Ycal, total_nlv=total_lv, number_splits=10, number_repetitions = cv_reps)
    
    rmsecv = cv_output[0]
    r2cv = cv_output[1]


    
    output = {'rmsecv':rmsecv,
              'rmseval': rmseval,
              'rmsep':rmsep,
             'r2cv':r2cv,
             'r2val':r2val,
             'r2p':r2p}
    
    return output
    


In [28]:
# --- get design and selected samples

design_df = pd.read_pickle(base_dir + "/experiments/" + experimentID + "/output/" + caseID_key + "_01_design_selected_samples.pkl")


In [29]:
# --- run all pls models

from datetime import datetime

print('start: ',datetime.now())


for jj in range(data_class.get_cal()["ycal"].shape[1]):
    
    print(data_class.y_names[jj])

    xx = data_class.get_cal()["xcal"]
    yy = np.ascontiguousarray(data_class.get_cal()["ycal"][:,jj:(jj+1)])
    xx_test = data_class.get_test()["xtest"]
    yy_test = np.ascontiguousarray(data_class.get_test()["ytest"][:,jj:(jj+1)])


    pls_performance = {}


    for ii in range(design_df.shape[0]):

        if ii%100==0:

            print(ii)


        current_run = dict(design_df.iloc[ii])

        selected_samples = current_run["selected_samples"]

        current_performance = model_performance_cv_val_test(xx,yy,selected_samples,xx_test,yy_test,total_lv=15, cv_reps = 1)

        current_run.update(current_performance)

        pls_performance[str(ii)] = current_run

    print('finish: ',datetime.now())


    # ¡¡¡ --- !!! ---> save output 

    df_output = pd.DataFrame.from_dict(pls_performance, orient="index")
    df_output.to_pickle(base_dir + "/experiments/" + experimentID + "/output/" + caseID_key + "_" + data_class.y_names[jj] + "_numba_02_pls_performance.pkl")

start:  2020-09-04 17:23:05.091489
moisture
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
finish:  2020-09-04 17:24:03.782008


In [36]:
df_output.iloc[0]["rmsecv"].shape

(15, 1, 1)

In [ ]:
# base_dir + "/experiments/python/2020-07-29_0001/output/" + caseID_key + "_" + data_class.y_names[0] + "_numba_02_pls_performance.pkl"